### Creación del modelo basado en contenido
Este notebook recorta el dataset del películas original, ya que no es factible calcular las similitudes entre cada par de películas con más de 45000 películas (requiere un espacio excesivo de memoria para nuestros equipos, aunque sea sólo en cálculos intermedios). Para recortar el datset nos quedamos con el 33% de las películas más populares (que más valoraciones tienen). Con las 14732 películas correspondientes a ese porcentaje sí que hemos podido trabajar sin problema.


A partir de este dataset se crea una tabla Nx(N-1) (en formato Numpy Array), cuya i-ésima fila consta de los IDs de las demás películas (todas salvo la número i) ordenados de mayor a menor similitud con la película i. Esta tabla constituye el modelo basado en contenido y se almacena en un fichero externo para ser utilizada por el sistema.

Importamos las librerías necesarias

In [14]:
import pandas as pd
import numpy as np
from ast import literal_eval
import pickle
from functools import partial

Cargamos y limpiar el dataset de películas según explican paso por paso los comentarios del código

In [3]:
#Cargamos el dataset
df_movies_original = pd.read_csv("movies_metadata.csv", low_memory=False)

#Mostramos sus dimensiones
print("Dimensiones originales:",df_movies_original.shape)

# En 'id' hay algunos valores no numéricos, descartamos esas filas
df_movies_original = df_movies_original[df_movies_original['id'].apply(lambda x: str(x).isdigit())]

# Hay ids repetidos en el dataset de películas. Nos quedamos sólo con uno de ellos
df_movies_original = df_movies_original.drop_duplicates(subset = ['id'])

# También ahy películas diferentes con títulos idénticos.Nos quedamos con solo una de ellas
df_movies_original = df_movies_original.drop_duplicates(subset = ['title'])

#Quitamos la película titulada "A movie" pues es un problema en el dialogo con el usuario
df_movies_original = df_movies_original[df_movies_original['title'] != 'A Movie']

# Fijamos un cuantil de 0.66 para quedarnos solo con el 33% de las películas más votadas
quantile = 0.66
# Calculamos el valor a partir del cual solo quedan por encima el 33% de las películas
vote_quantile = df_movies_original["vote_count"].quantile(quantile)

# Mostramos cual el número de votos de corte a partir del cuál sí tomamos películas
print("Cuantil de voto", str(quantile) + ":", vote_quantile)

# Nos quedamos sólo con ese 33% de películas
df_movies_original = df_movies_original.loc[df_movies_original["vote_count"] >= vote_quantile]

#Mostramos las dimensiones después de quitar películas
print("Dimensiones tras quitar películas:", df_movies_original.shape)

#Dejamos en el dataset los géneros de las películas en forma de lista para trabajar más cómodamente con ellos
df_movies_original["genres"] = df_movies_original["genres"].apply(literal_eval)
df_movies_original["genres"] = df_movies_original["genres"].apply(lambda genres: list(map(lambda genre: genre["name"], genres)))

# Guardamos el dataset "limpio" en el fichero "movies_catalog_clean.csv"
df_movies_original.to_csv("movies_catalog_clean.csv")

Dimensiones originales: (45466, 24)
Cuantil de voto 0.66: 20.0
Dimensiones tras quitar películas: (14732, 24)


Cargamos y limpiamos la tabla con los créditos de las películas (tiene información relevante para nosotros como el reparto o el director)

In [4]:
df_credits_original = pd.read_csv("credits.csv")

# Eliminamos también repeticiones de ids en los créditos de las películas
df_credits_original = df_credits_original.drop_duplicates(subset = ['id'])

df_credits_original.to_csv("credits_clean.csv")

Cargamos las tablas limpias que hemos guardado en el fichero para comprobar que todo está bien y trabajar sobre ellas

In [5]:
df_movies = pd.read_csv("movies_catalog_clean.csv")
df_credits = pd.read_csv("credits_clean.csv")

Hacemos un join entre la tabla con las películas y la que tiene sus créditos según el identificador de película. utilizaremos variables que provienen de ambas tablas y necesitamos primero su correspondencia.

In [6]:
df = df_movies.merge(df_credits, on ='id')

En el modelo, tendremos también en cuenta palabras clave sobre la película. Estas palabras clave están en un tercer dataset que cargamos, limpiamos y unimos a los dos anteriores (de nuevo un join) a continuación.

In [7]:
# Eliminamos IDs repetidos (los hay)
df_keywords = pd.read_csv("keywords.csv").drop_duplicates(subset = ['id'])
df = df.merge(df_keywords, on ='id')

Convertimos las columnas que vamos a usar en el modelo a objetos python mediante la función "literal_eval"

In [8]:
features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    df[feature] = df[feature].apply(literal_eval)

Creamos una función  que, dado el equipo de la película, extrae a los directores (puede haber varios al existir codirectores). Y otra función que datda una lista de lista de personas devuelve una lista con los n primeros nombres de las mismas.

In [11]:
def director(crew):
    for person in crew:
        if person['job'] == 'Director':
            return [person['name']]
    return np.nan

def getTop(n, mList):
    if isinstance(mList, list):
        names = list(map(lambda person: person["name"], mList))
        names = names[:n]
        return names
    #Devolvemos la lista vacía en caso de datos incorrectos
    return []

Observamos la salida al extraer el director de cada película

In [12]:
df['crew'].apply(director)

0          [John Lasseter]
1           [Joe Johnston]
2          [Howard Deutch]
3        [Forest Whitaker]
4          [Charles Shyer]
               ...        
14727      [Colin Minihan]
14728         [Beth David]
14729         [Larry Shaw]
14730     [Georges Méliès]
14731     [Georges Méliès]
Name: crew, Length: 14732, dtype: object

Utilizando las funciones anteriores modificamos el dataset para crear una columna "director" con los nombres de los directores de la película, dejar en la columna "cast" los 3 actores principales de la película (el orden de aparición en el dataset sabemos que es de mas a menos importante para la película), dejar en la columna "keywords" las 10 palabras clave principales y en la columna "genres" los hasta 10 géneros principales de la película (en general no tienen tantos).

In [15]:
dfContentBased = pd.DataFrame()
dfContentBased['id'] = df['id']
dfContentBased['director'] = df['crew'].apply(director)

featuresTop = [('cast', 3), ('keywords', 10)]
for feature, topLimit in featuresTop:
    dfContentBased[feature] = df[feature].apply(partial(getTop, topLimit))
    
dfContentBased["genres"] = df["genres"].apply(lambda l: l[:10])

Creamos una función que compacta los datos convirtiéndolos a minúscula y eliminar especios entre palabras. **No queremos que al calcular similitudes el director John Lasseter y John Waters en común el término John, pues son personas distintas**. Para que los términos sean los directores completos (e igual con actores, palabras clave o género) eliminamos los espacios. 

In [20]:
def compact_data(data):
    if isinstance(data, list):
        return [str.lower(element.replace(" ", "")) for element in data]
    else:
        return ""
    
features = ["director", "cast", "keywords", "genres"]
for feature in features:
    dfContentBased[feature] = dfContentBased[feature].apply(compact_data)

Observamos el resultado de las primeras filas tras compactar los datos

In [21]:
dfContentBased.head(5)

,id,director,cast,keywords,genres
0,862,[johnlasseter],"[tomhanks, timallen, donrickles]","[jealousy, toy, boy, friendship, friends, riva...","[animation, comedy, family]"
1,8844,[joejohnston],"[robinwilliams, jonathanhyde, kirstendunst]","[boardgame, disappearance, basedonchildren'sbo...","[adventure, fantasy, family]"
2,15602,[howarddeutch],"[waltermatthau, jacklemmon, ann-margret]","[fishing, bestfriend, duringcreditsstinger, ol...","[romance, comedy]"
3,31357,[forestwhitaker],"[whitneyhouston, angelabassett, lorettadevine]","[basedonnovel, interracialrelationship, single...","[comedy, drama, romance]"
4,11862,[charlesshyer],"[stevemartin, dianekeaton, martinshort]","[baby, midlifecrisis, confidence, aging, daugh...",[comedy]


A continuación concatenamos, separando con espacios, las distintas variables que representarán a la película al calcular similitudes de nuestra películas.

In [22]:
def joinFeatures(features, dataList):
    joinedResult = ""
    for feature in features:
        joinedResult += " ".join(dataList[feature]) + " "
    # Quitamos el último espacio, que sobra al no separar palabras
    joinedResult = joinedResult[:-1]
    return joinedResult

dfContentBased['featuresSoup'] = dfContentBased.apply(partial(joinFeatures, features), axis=1)

Mostramos el resultado para la primera película y vemos cómo la frase que la representa tiene en cuenta su director, sus actores principales, algunas palabras clave que la describen y los géneros a los que pertenece.

In [24]:
dfContentBased['featuresSoup'][0]

'johnlasseter tomhanks timallen donrickles jealousy toy boy friendship friends rivalry boynextdoor newtoy toycomestolife animation comedy family'

Vectorizamos las descripciones que acabamos de crear para las películas de nuestro dataset. **Como medida, utilizamos la frecuencia de aparición (CountVectorized) en lugar de TF-IDF, porque no queremos penalizar que un director o un actor aparezca en muchas películas**. Si un director o un actor aparece en muchas películas será que es muy famoso y relevante para la comparación; no tiene sentido introducir el factor IDF que lo penalice.

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
countMatrix = CountVectorizer().fit_transform(dfContentBased['featuresSoup'])

Calculamos la similitud entre cada par de vectores de películas k-dimensionales (siendo k el tamaño del vocabulario resultante).

In [23]:
from sklearn.metrics.pairwise import cosine_similarity
simMatrix = cosine_similarity(countMatrix, countMatrix)

Ordenamos los valores de similitud para cada fila de la matriz de mayor a menor y los sustituimos por el índice que ocupaban antes de ordenar la matriz (simMatrix, axis=1), en todas las filas eliminamos el primera valor (la similitus máxima=1.0 siempre la tenemos con nosotros mismos).

In [45]:
recommendIndexes = np.argsort(simMatrix, axis=1)
recommendIndexes = recommendIndexes[:, -2::-1]
recommendIndexes

array([[ 8453,  2046, 11452, ...,  7983,  7981, 14731],
       [12248, 11189,  6747, ...,  7871,  7870,  7365],
       [ 2258,  1148,  1011, ...,  5894,  5895, 14731],
       ...,
       [ 4038, 12081, 14386, ..., 11723,  6780, 14731],
       [11887, 14715, 14731, ..., 11341, 11339,  4807],
       [14730, 14632, 14715, ...,  8730,  8729,     0]], dtype=int64)

En lugar de posiciones en el array, queremos tener los IDs de las películas correspondientes a dichas posiciones. Hacemos un mapeo en la matriz por filas para que en lugar de 8453 ponga 10193, siendo 10193 el ID de la película que está en la posición 8453 de la matriz.

In [53]:
dfContentBased['id'][recommendIndexes[0]].values

array([ 10193,    863, 256835, ...,  18533,  17336,  49280], dtype=int64)

In [56]:
idsMaxToMin = np.array(list(map(lambda indexes: dfContentBased['id'][indexes].values, recommendIndexes)))
idsMaxToMin

array([[ 10193,    863, 256835, ...,  18533,  17336,  49280],
       [262788,  25475,   9992, ...,  11914,    761,   9029],
       [ 11520,  27472,  18080, ...,     26,   2026,  49280],
       ...,
       [ 18736,  50794, 444706, ..., 309299,  10075,  49280],
       [  2963, 104700,  49280, ..., 246860, 253295,  29455],
       [ 49279, 104466, 104700, ...,  46138,  16987,    862]], dtype=int64)

Guardamos la matriz en el fichero externo "contentBasedModel.npy"

In [57]:
from numpy import save
save('contentBasedModel.npy', idsMaxToMin)